In [1]:
!pip install -U pip
!pip install -U pandas
!pip install -U sklearn
!pip install -U imbalanced-learn

Requirement already up-to-date: pip in /home/admin/venv-jupyter/lib/python3.7/site-packages (20.1)
Requirement already up-to-date: pandas in /home/admin/venv-jupyter/lib/python3.7/site-packages (1.0.3)
Requirement already up-to-date: sklearn in /home/admin/venv-jupyter/lib/python3.7/site-packages (0.0)
Requirement already up-to-date: imbalanced-learn in /home/admin/venv-jupyter/lib/python3.7/site-packages (0.6.2)


In [2]:
import pandas as pd 
import sys

from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

import classifier_tool as tool

In [3]:
label = 'angin_topan'
rep = 't_swrem_stem'

clf_list = ['SVC', 'LinearSVC', 'MultinomialNB', 'BernoulliNB', 'LogisticRegression', 'RandomForestClassifier',]
f_gold = '/home/admin/text_processing/anotated_data/classifier_gold_standard/preprocessed/sample_angin_topan_gold_dataset_pre.json'
df_gold = pd.read_json(f_gold)

In [4]:
def test_clf(rep, df, clf, fold=10):
    X, vectorizer = tool.construct_bow_unigrams(df[rep])
    y = df['label']
      
    report = tool.eval_cv(fold, X, y, clf)
    y_report = [ d['Y'] for d in report ]
    n_report = [ d['N'] for d in report ]
    
    df_y = pd.DataFrame(y_report)
    df_n = pd.DataFrame(n_report)
    df_y.loc['avg'] = df_y.mean()
    df_n.loc['avg'] = df_n.mean()
    df_y_avg = df_y.loc[['avg']]
    df_y_avg = df_y_avg.rename(columns={'precision' : 'y_prec', 'recall':'y_recall', 'f1-score' : 'y_f1', 'support' : 'y_support'})
    
    return df_y_avg, df_y, df_n

In [5]:
def get_classifier(name):
    clf = None
    
    if 'SVC' == name:
        clf = SVC()
    elif 'LinearSVC' == name:
        clf = LinearSVC()
    elif 'MultinomialNB' == name:
        clf = MultinomialNB()
    elif 'BernoulliNB' == name:
        clf = BernoulliNB()
    elif 'LogisticRegression' == name:
        clf = LogisticRegression(random_state=0)
    elif 'RandomForestClassifier' == name:
        clf = RandomForestClassifier(max_depth=100, random_state=0, n_estimators=100)
    
    return clf

In [6]:
df_summary = pd.DataFrame()

for cl in clf_list:
    clf = get_classifier(cl)
    df_y_avg, df_y, df_n = test_clf(rep, df_gold, clf)
    df_summary[cl] = df_y_avg.loc['avg']
df_summary

,SVC,LinearSVC,MultinomialNB,BernoulliNB,LogisticRegression,RandomForestClassifier
y_prec,0.870675,0.756896,0.518011,0.764915,0.844537,0.804712
y_recall,0.603727,0.714747,0.766776,0.467682,0.655912,0.650131
y_f1,0.707541,0.731171,0.610617,0.575365,0.736328,0.714496
y_support,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000
